In [1]:
import bz2
import numpy as np
import re
from keras.preprocessing import text, sequence

Using TensorFlow backend.


In [2]:
from gensim.models import Word2Vec
# importing all necessary modules 
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gmsan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('test.ft.txt.bz2')

In [6]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [5]:

train_text = [word_tokenize(text) for text in train_texts]

print(len(train_text))
print(len(train_text[0]))

KeyboardInterrupt: 

In [7]:
max_features = 20000
maxlen = 100
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_texts)
tokenized_train = tokenizer.texts_to_sequences(train_texts)
tokenized_test = tokenizer.texts_to_sequences(test_texts)


In [8]:

X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [26]:
test_labels[0]

1

In [25]:
EMBEDDING_DIM = 200

In [26]:
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

#GloVes Load

Glove = 'glove.6B.200d.txt'

glove2word2vec(glove_input_file=Glove, word2vec_output_file="gensim_glove_vectors.txt")
embeddings = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0

for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
        
del embeddings

In [19]:
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [20]:
def build_model():
    sequences = layers.Input(shape=(maxlen,))
    embedded = layers.Embedding(max_features, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
model = build_model()

In [27]:
history = model.fit(
    X_train, 
    train_labels, 
    batch_size=1024,
    epochs=10,
    validation_split=0.2 )

Train on 2880000 samples, validate on 720000 samples
Epoch 1/10
2880000/2880000 [==============================] - 1257s 436us/sample - loss: 0.2202 - binary_accuracy: 0.9100 - val_loss: 0.1887 - val_binary_accuracy: 0.9264
Epoch 2/10
2880000/2880000 [==============================] - 1166s 405us/sample - loss: 0.1772 - binary_accuracy: 0.9315 - val_loss: 0.1841 - val_binary_accuracy: 0.9290
Epoch 3/10
2880000/2880000 [==============================] - 1163s 404us/sample - loss: 0.1624 - binary_accuracy: 0.9384 - val_loss: 0.1852 - val_binary_accuracy: 0.9289
Epoch 4/10
2880000/2880000 [==============================] - 1164s 404us/sample - loss: 0.1515 - binary_accuracy: 0.9433 - val_loss: 0.1936 - val_binary_accuracy: 0.9266
Epoch 5/10
2880000/2880000 [==============================] - 1165s 404us/sample - loss: 0.1427 - binary_accuracy: 0.9472 - val_loss: 0.1868 - val_binary_accuracy: 0.9284
Epoch 6/10
2880000/2880000 [==============================] - 1808s 628us/sample - loss: 0.1

In [28]:
model.save('glove_model.h5')


In [16]:
print(len(tokenizer.word_index) + 1)

905946


In [17]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

ImportError: cannot import name 'run_classifier' from 'bert' (unknown location)

In [14]:
pip install bert-tensorflow


Note: you may need to restart the kernel to use updated packages.


In [11]:

# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'D:\UWaterloo\SPRING2020\ECE657\project\bert\'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
#   from google.colab import auth
#   auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

905946